# Camada Convolucional

Como vimos, as camadas densas tem severos problemas, principalmente para trabalhar com imagens, pois a dimensionalidade da entrada é muito grande o que implica num alto número de parâmetros para se otimizar.

Contornando essa situação, temos as camadas convolucionais.
Ao longo do tempo, várias tipos de camadas convolucionais foram criadas.
Nesta aula veremos algumas dessas camadas.


Antes de começar, vamos instalar o Pytorch. Esse pequeno bloco de código abaixo é usado somente para instalar o Pytorch para CUDA 10. Execute esse bloco somente uma vez e ignore possíveis erros levantados durante a instalação.

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [1]:
!pip3 install torch torchvision

In [0]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim

import time, os, sys, numpy as np

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

In [0]:
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## Camada Convolucional

A [camada convolucional](https://beta.mxnet.io/api/gluon/_autogen/mxnet.gluon.nn.Conv2D.html) é considerado o principal módulo das rede convolucionais (*ConvNet*) pois é responsável por fazer a maior parte do trabalho, ou seja, aprendar filtros que extraem características.

Tecnicamente, uma camada convolucional é composta por um conjunto de filtros (ou *kernels*) aprendíveis (que, na verdade, representam os parâmetros dessa camada).
Geralmente, cada um desses filtros é relativamente pequeno (em termos de largura e altura), mas se estende por toda a profundidade dos dados de entrada.
Por exemplo, cada filtro em uma primeira camada de uma *ConvNet* geralmente tem tamanho $3 \times h\times w$, onde $h$ e $w$ são altura e largura, respectivamente, e o $3$ representa a profundidade dos filtros que, neste caso, são ligados aos canais de cores da entrada (RGB).
Num primeiro momento, trabalharemos somente com filtros bidimensionais pois deixaremos de lado a dimensão relacionada aos canais.
Durante o *forward*, convolucionamos cada filtro sobre a entrada calculando o produto entre esses valores e gerando, como saída, um mapa de ativação bidimensional que fornece as respostas desse filtro em todas as possíveis posições.
Em outras palavras, cada filtro pode ser visto como um neurônio que irá combinar os canais de entradas gerando uma saída.
Intuitivamente, a rede aprenderá filtros que são ativados quando encontram algum tipo de características visual interessante, como uma borda na primeira camada, ou eventualmente padrões mais complexos nas camadas mais finais da rede.
Cada camada convolucional é composta por conjuntos inteiro de filtros onde, cada um deles, produzirá um mapa de ativação bidimensional separado.
Esses mapas de ativação são empilhados (na profundidade) produzindo a saída final (comumente chamada de *feature maps*).

Formalmente, o procedimento de convolução 2-D, comumente empregado nesse tipo de camada, recebe uma entrada de duas dimensões $x$ e um vetor de peso 2-D $K$ (nesse caso, com tamanho $n\times n$) e os processa da seguinte forma:

$$ Y[k, l] = \sum_{i=1}^{n} \sum_{j=1}^{n} X(k + i - 1, l + j -1) K(i,j) $$
, onde $Y$ é a saída ou *feature map*.

Abaixo temos um exemplo de convolução.
Neste caso, a entrada é uma matriz bidimensional com uma altura de 3 e largura de 3 ($3 \times 3$ e o filtro tem dimensões $2\times 2$.
Este exemplo destaca (em azul) um passo do processo de convolução.
Entretanto, como dito anteriormente, a janela (filtro) de convolução percorre toda a entrada ao longo da altura e largura, sempre multiplicando e somando os elementos da entrada pelos valores do filtro.

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1JSTFGFpfjdMVvkaHUUwzGlgle6UVhFjo">
</p>

O exemplo acima temos somente **um** canal de entrada e de saída. Porém, como funciona a convolução com múltiplos canais de entrada e saída?

### Múltiplos canais de entrada e saída


No exemplo anterior, tanto os filtros quanto as saídas podem ser vistas como matrizes bidimensionais.
Entretanto, quando adicionamos mais canais, entradas, filtros, e saídas passam a ser representadas como matrizes tridimensionais.
Por exemplo, cada imagem de entrada RGB tem a forma $3 \times h \times w$, onde $h$ e $w$ são a altura e largua respectivamente..
A dimensão com um tamanho de 3 é referenciada como a dimensão do canal.

#### Múltiplos Canais de Entrada

Quando os dados de entrada contêm múltiplos canais, precisamos de filtros de convolução com o mesmo número de canais que os dados de entrada, para que ele possamos processar uma convolução.
Assumindo que o número de canais para os dados de entrada é $c_i$, o número de canais dos filtro de convolução também precisa ser $c_i$.
Quando $c_i = 1$, o filtro de convolução é uma uma matriz bidimensional com dimensões $k_h \times k_w$.

No entanto, quando $ c_i> 1$, precisamos de um filtro que contenha uma matriz $k_h \times k_w$ **para cada canal de entrada**.
Concatenanado todos esses $c_i$ arrays juntos geramos um filtro de convolução com dimensões $c_i \times k_h \times k_w$.
Como a entrada e o filtro da convolução tem cada um $c_i$ canais, podemos criar uma correspondência entre canais para executar a operação de convolução.
Em outras palavras, podemos fazer a convolução da matriz bidimensional da entrada com o *kernel* bidimensional para cada canal, somando os resultados ao longo dos canais $c_i$, produzindo uma só matriz bidimensional de saída.

Na figura abaixo, demonstramos um exemplo de convolução com dois canais de entrada.
As partes sombreadas representam o primeiro elemento de saída, bem como os elementos de entrada e de matriz do kernel usados em sua computação: $(1 \times1 + 2 \times 2 + 4 \times 3 + 5 \times 4) + (0 \times 0 + 1 \times 1 + 3 \times 2 + 4 \times 3) = 56$.

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1OC-o75LRvxi8KllotBoR8cS38NS4EOTm">
</p>

### Múltiplos canais de saída

Independentemente do número de canais de entrada, até agora nós sempre acabamos com um canal de saída.
No entanto, aumentar o número de canais (neurônios) em cada camada implica em aumentar a poder de representação daquela camada.
Nas arquiteturas de redes neurais mais populares, na verdade aumentamos a dimensão do canal à medida que avançamos na rede neural, geralmente diminuindo a resolução. 

Denote por $c_i$ e $c_o$ o número de canais de entrada e saída, respectivamente, e deixe que $k_h$ e $k_w$ sejam a altura e a largura do filtro convolucional.
Para obter uma saída com múltiplos canais, podemos criar uma matriz de kernel de forma $c_i \times k_h \times k_w $ para cada canal de saída $c_o$ (que também pode ser visto como neurônio).
Concatenamos na dimensão do canal de saída, para que o filtro de convolução tenha resolução final de $c_o \times c_i \times k_h \times k_w$.
Nas operações de convolução, o resultado em cada canal de saída é calculado a partir do filtro de convolução correspondente a esse canal de saída e recebe a entrada de todos os canais na matriz de entrada.

Para ter uma visão mais ampla da convolução em múltiplos canais de entrada e saída, acesse esse [site](http://cs231n.github.io/convolutional-networks/) e procurem pela gif relacionada ao tema.

### Pytorch e o caso de estudo LeNet-5

Frameworks modernos implementam camadas convolucionais de forma fácil e intuitiva.
No Pytorch, a [camada de convolução](https://pytorch.org/docs/stable/nn.html#conv2d) tem alguns dos parâmetros que vimos anteriormente na sua declaração.

Vamos implementar uma rede baseada na [LeNet-5](https://ieeexplore.ieee.org/document/726791) e entender cada parâmetro.
A rede tem essa arquitetura (ignorem, nesse primeiro momento, os *subsamplings*):

<p align="center">
  <img width=700 src="https://miro.medium.com/max/2625/1*1TI1aGBZ4dybR6__DI9dzA.png">
</p>

As camadas com a letra C, são convoluções. Camadas que começam com a letra S, são *subsampling* e devem ser ignoradas neste primeiro momento.
Já camada com ínicio F, são *fully-connected*.
Abaixo, uma tabela que compila toda a configuração da rede.

<p align="center">
  <img width=700 src="https://engmrk.com/wp-content/uploads/2018/09/LeNEt_Summary_Table.jpg">
</p>

Abaixo, recriamos a rede no Pytorch sem as camadas de *subsampling*.


In [0]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.01)
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.01)   

In [5]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.1, 128, 0.000001

# rede baseada na LeNet-5 
net = nn.Sequential(
nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5),     # entrada: 1 canal e saida: 6 canais
        nn.Tanh(),  
        nn.Conv2d(in_channels=6,out_channels= 16, kernel_size=5),    # entrada: 6 canais e saida: 16 canais
        nn.Tanh(),  
        nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5),   # entrada: 16 canais e saida: 120 canais
        nn.Tanh(),  
        nn.Flatten(),  # lineariza formando um vetor        # entrada: 120 canais e saida: linear
        nn.Linear(48000, 84),
        nn.Tanh(),  
        nn.Linear(84, 10)
     ) 

# Não é necessário inicializar 
# net.apply(weights_init)

# Sending model to device
net.to(device)

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do pytorch
trainer = optim.SGD(net.parameters(), lr=0.01, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

0it [00:00, ?it/s]

26427392it [00:02, 13155736.85it/s]                             


Extracting /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 95265.10it/s]                            
0it [00:00, ?it/s]

...

4423680it [00:01, 3921171.63it/s]                             
0it [00:00, ?it/s]

Extracting /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw


8192it [00:00, 31594.12it/s]            

Extracting /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw
Processing...
Done!
training on cuda


epoch 1, train loss 0.9058, train acc 0.701, test loss 0.6417, test acc 0.762, time 14.5 sec
epoch 2, train loss 0.5700, train acc 0.790, test loss 0.5495, test acc 0.797, time 14.0 sec
epoch 3, train loss 0.5106, train acc 0.815, test loss 0.5151, test acc 0.811, time 14.2 sec
epoch 4, train loss 0.4749, train acc 0.830, test loss 0.4895, test acc 0.826, time 14.1 sec
epoch 5, train loss 0.4485, train acc 0.840, test loss 0.4661, test acc 0.829, time 14.3 sec
epoch 6, train loss 0.4255, train acc 0.849, test loss 0.4454, test acc 0.840, time 14.0 sec
epoch 7, train loss 0.4080, train acc 0.856, test loss 0.4375, test acc 0.842, time 14.0 sec
epoch 8, train loss 0.3916, train acc 0.861, test loss 0.4344, test acc 0.843, time 14.0 sec
epoch 9, train loss 0.3782, train acc 0.866, test loss 0.4039, test acc 0.853, time 14.0 sec
epoch 10, train loss 0.3651, train acc 0.870, test loss 0.4086, test acc 0.850, time 14.0 sec
epoch 11, train loss 0.3549, train acc 0.874, test loss 0.3890, test 

### Hiper-parâmetros: *Padding* e *Stride*

No exemplo anterior, a entrada tinha dimensões $6\times8$ e o filtro de convolução $1\times2$.
O processamento da convolução produziu, então, uma saída com uma resolução $6\times7$. 
Esse diferença da resolução é motivada pelo próprio processamento da convolução e da sua forma de lidar com as extremidades da imagem.
Em geral, assumindo que a entrada tem tamanho $n_h\times n_w$ e o filtro de convolução tem dimensões $k_h \times k_w$, então o tamanho da saída pode ser calculado da seguinte forma:

$$ (n_h-k_h + 1) \times (n_w-k_w + 1)$$

Neste caso, as dimensões da saída é determinada pelos tamanhos da entrada e do filtro de convolução.

Em alguns casos, podemos incorporar técnicas comum de processamento de imagem (como *Padding* e *Stride*) que afetam diretamente o tamanho da saída:

* Em geral, como filtros geralmente têm dimensões maiores que 1, após muitas convoluções sucessivas, a saída termina ficando muito menor do que a entrada.
Por exemplo, imagine que uma imagem de entrada com $240\times 240$ pixels seja processada por 10 camadas de convoluções $5\times 5$.
Neste caso, a imagem inicial será reduzida para uma saída de $200\times 200$ pixels, ou seja,  30% da imagem original é eliminanda e, com ela, informações interessante próxima das extremidades da imagem de entrada. *Padding* lida com esse problema. 
* Em alguns casos, como quando a entrada tem uma resolução muito grande, queremos reduzir drasticamente a resolução da imagem durante o seu processamento. *Strides* podem ajudar nesses casos.

#### *Padding*

Como descrito acima, um problema complicado ao se trabalhar com camadas convolucionais é a perda de pixels (e, consequentemente, informação) na extremidade da imagem.
Como normalmente usamos filtros pequenos, a perda num geral é pequena.
Entretanto, ela se torna maior à medida que aplicamos várias camadas convolucionais sucessivas.
Uma solução direta para esse problema é adicionar pixels extras ao redor da imagem de entrada, de forma a aumentar o tamanho efetivo da imagem.
Esse processo é conhecido como *padding*.
Normalmente, definimos os valores desses pixels extras como 0 (*zero-padding*).
Abaixo, temos um exemplo visual de uma entrada $3\times 3$ com *padding* de tamanho 1 em todos os lados sendo processada por um *kernel* $2\times 2$.

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1Y66FJerJtlyQZGNgf_JzG5LLPsYrmREH">
</p>

Para ficar ainda mais claro o funcionamento do *padding*, implementamos esse processo usando o trecho de código abaixo.
Neste caso, a entrada, de tamanho $3\times 3$, tem sua resolução aumentada  $5\times 5$ usando *padding*.
Dessa forma, a saída correspondente é também aumentada para $4\times 4$.

In [6]:
n = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([0.1]))
X = n.sample((1,1,3,3))
# adicionando o padding
X_pad = F.pad(X, mode='constant', pad=(0,0,1,1,1,1,0,0,0,0), value=0)  # Faz 0 pad da última dimension, 1 pad pra cada lada da penúltimo dimension
                                                                       # 1 pad pra cada lado da anti-penúltima dimension e 0 para outras

conv2d = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=2)

print('-----Entrada original-----')
print(torch.squeeze(X))
print('\n-----Entrada Padding-----')
print(torch.squeeze(X_pad))

print('\n-----Saida com entrada original-----')
print(conv2d(X.view(1,1,3,3)))
print('\n-----Saida com entrada padding-----')
print(conv2d(X_pad.view(1,1,5,5)))

-----Entrada original-----
tensor([[ 0.0130,  0.0105,  0.1495],
        [-0.0895,  0.1255,  0.0794],
        [ 0.0396, -0.1081, -0.1174]])

-----Entrada Padding-----
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0130,  0.0105,  0.1495,  0.0000],
        [ 0.0000, -0.0895,  0.1255,  0.0794,  0.0000],
        [ 0.0000,  0.0396, -0.1081, -0.1174,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

-----Saida com entrada original-----
tensor([[[[0.4509, 0.3543],
          [0.4444, 0.5421]]]], grad_fn=<MkldnnConvolutionBackward>)

-----Saida com entrada padding-----
tensor([[[[0.4801, 0.4782, 0.4215, 0.4522],
          [0.5161, 0.4509, 0.3543, 0.4947],
          [0.5125, 0.4444, 0.5421, 0.5259],
          [0.4663, 0.5449, 0.5228, 0.4643]]]],
       grad_fn=<MkldnnConvolutionBackward>)


Em geral, se adicionarmos um total de $p_h$ linhas de *padding* (aproximadamente metade na parte superior e metade na parte inferior) e um total de $p_w$ colunas de *padding* (aproximadamente metade à esquerda e metade à direita da entrada), as dimensões da saída serão calculadas da seguinte forma:

$$ (n_h-k_h + p_h + 1) \times (n_w-k_w + p_w + 1) $$

Isso significa que a altura e a largura da saída aumentarão em $p_h$ e $p_w$, respectivamente.

Em muitos casos, definiremos $p_h = k_h-1$ e $p_w = k_w-1$ para termos a entrada e saída com as mesmas dimensões.
Isso facilitará o cálculo da dimensão da saída de cada camada ao construir a rede.
Assumindo que $k_h$ é ímpar, podemos preencher $p_h/2$ linhas nos dois lados da altura.
Se $k_h$ for par, uma possibilidade é preencher $\lceil p_h/2 \rceil $ linhas na parte superior da entrada e $\lfloor p_h/2 \rfloor$ linhas na parte inferior.
A largura é tratada da mesma maneira.

Redes neurais convolucionais comumente usam filtros convolucionais com valores ímpares de altura e largura, como 1, 3, 5 ou 7.
Escolher tamanhos ímpares de *kernel* tem o benefício de preservar a dimensionalidade espacial em relação ao *padding*, ou seja, o mesmo número de linhas e colunas serão adicionadas em todos os lados da entrada.

No exemplo a seguir, criamos uma camada convolucional com filtro de altura e largura iguais à 3 e aplicamos *padding* de 1 pixel em todos os lados do dado de entrada.
Logo, dada uma entrada com resolução $8\times 8$, temos que a altura e a largura da saída também serão 8.

In [7]:
n = torch.distributions.Uniform(torch.tensor([0.0]),torch.tensor([1.0]))
X = n.sample((8,8))

# Por conveniência, definimos uma função para calcular a camada convolucional.
# Esta função inicializa os pesos da camada convolucional e executa
# modificacoes correspondentes de dimensionalidade na entrada e saída
def comp_conv2d(conv2d, X):
    # (1,1) indica o tamanho do batch e a quantidade de canais
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X.view(1,1,8,8))
    # exclui as duas primeiras dimensoes que nao nos interessam
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=3, padding=0)
print(comp_conv2d(conv2d, X).shape)
  
# Note que aqui 1 linha ou coluna é coloca em ambos os lados,
# então um total de 2 linhas ou colunas são adicionadas
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
print(comp_conv2d(conv2d, X).shape)

torch.Size([6, 6])
torch.Size([8, 8])


Quando a altura e a largura do kernel de convolução são diferentes, podemos fazer com que a saída e a entrada tenham a mesma altura e largura definindo diferentes valores de *padding*  para altura e largura.

In [8]:
# Aqui, usamos um kernel de convolução com uma altura de 5 e uma largura de 3
# O *padding* na dimensão da altura eh 2 enquanto que,
# na dimensao da largura, eh 1
conv2d = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

#### *Stride*

Ao calcular convolução, começamos com a janela (filtro) no canto superior esquerdo da matriz de entrada, e, em seguida, o deslizamos por todos os locais, para baixo e para a direita.
Anteriormente, sempre realizamos um passo de um pixel por vez.
No entanto, às vezes, seja por eficiência computacional ou porque queremos reduzir o tamanho da entrada, podemos mover a janela mais de um pixel de cada vez,  pulando e igorando muitos pixels de uma vez.

Referimo-nos ao número de linhas e colunas percorridas por um passo como *stride*.
Até agora, usamos *strides* de 1, tanto para altura quanto largura.
Às vezes, podemos querer usar um valor maior de *stride*.
Logicamente, que o valor do *stride* impacta diretamente na saída.
Em geral, quando o *stride* para a altura é $s_h$ e o *stride* para a largura é $s_w$, o tamanho da saída será calculada da seguinte forma:

$$\lfloor (n_h-k_h + p_h + s_h) / s_h \rfloor \times \lfloor(n_w-k_w + p_w + s_w) / s_w \rfloor$$

Se definirmos $p_h = k_h-1$ e $p_w = k_w-1$, então o cálculo da saída será simplificado para $\lfloor (n_h + s_h-1) /s_h \rfloor \times \lfloor(n_w + s_w-1) /s_w \rfloor$.

A figura abaixo mostra uma operação de convolução com *stride* de 3 verticalmente e 2 horizontalmente.
Podemos ver que quando o segundo elemento da primeira coluna é gerado, a janela de convolução desliza três linhas.
A janela de convolução desliza duas colunas para a direita quando o segundo elemento da primeira linha é gerado..

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1roGo9atDF5l2ocf9as0L_FDeMDxnAyuL">
</p>

Abaixo, um exemplo comparando o tamanho da saída quando usamos *stride* (1, 1), (2, 2), e (3, 4) para altura e largura, respectivamente.
Notem que, com a mesma entrada X de tamanho $8\times8$ a saída fica bem diferente.

In [9]:
n = torch.distributions.Uniform(torch.tensor([0.0]),torch.tensor([1.0]))
X = n.sample((8,8))

conv2d = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=3, padding=1, stride=1)
print('-----Stride 1, Padding 1-----')
print(comp_conv2d(conv2d, X).shape)

conv2d = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=3, padding=1, stride=2)
print('\n-----Stride 2, Padding 1-----')
print(comp_conv2d(conv2d, X).shape)

conv2d = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
print('\n-----Stride (3,4), Padding (0,1)-----')
print(comp_conv2d(conv2d, X).shape)

-----Stride 1, Padding 1-----
torch.Size([8, 8])

-----Stride 2, Padding 1-----
torch.Size([4, 4])

-----Stride (3,4), Padding (0,1)-----
torch.Size([2, 2])


### Continuação: Pytorch e o caso de estudo LeNet-5

Vamos agora definir explicitamente o *padding* e *stride* em cada camada convolucional.
Como, por padrão, o *padding* e *stride* dessa camada é 0 e 1, o nosso resultado ainda não vai mudar. Entretanto, agora podemos calcular a saída exata de cada camada.

<p align="center">
  <img width=700 src="https://miro.medium.com/max/2625/1*1TI1aGBZ4dybR6__DI9dzA.png">
</p>


<p align="center">
  <img width=700 src="https://engmrk.com/wp-content/uploads/2018/09/LeNEt_Summary_Table.jpg">
</p>


In [10]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 10, 0.1, 128, 0.000001

# rede baseada na LeNet-5 
net = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5, stride=1, padding=0),     # entrada: (b, 1, 32, 32) e saida: (b, 6, 28, 28)
        nn.Tanh(),
        nn.Conv2d(in_channels=6,out_channels=16, kernel_size=5, stride=1, padding=0),    # entrada: (b, 6, 28, 28) e saida: (b, 16, 24, 24)
        nn.Tanh(),
        nn.Conv2d(in_channels=16,out_channels=120, kernel_size=5, stride=1, padding=0),   # entrada: (b, 16, 24, 24) e saida: (b, 120, 20, 20)
        nn.Tanh(),
        nn.Flatten(),  # lineariza formando um vetor                              # entrada: (b, 120, 20, 20) e saida: (b, 120*20*20) = (b, 48000)
        nn.Linear(48000, 84),                                          # entrada: (b, 48000) e saida: (b, 84)
        nn.Tanh(),
        nn.Linear(84, 10))                                                             # entrada: (b, 84) e saida: (b, 10)

# Sending model to device
net.to(device)

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

training on cuda
epoch 1, train loss 0.5789, train acc 0.788, test loss 0.4645, test acc 0.828, time 14.1 sec
epoch 2, train loss 0.4043, train acc 0.855, test loss 0.3929, test acc 0.857, time 14.1 sec
epoch 3, train loss 0.3563, train acc 0.872, test loss 0.3747, test acc 0.864, time 14.1 sec
epoch 4, train loss 0.3224, train acc 0.883, test loss 0.3545, test acc 0.870, time 14.0 sec
epoch 5, train loss 0.2977, train acc 0.891, test loss 0.3609, test acc 0.867, time 14.1 sec
epoch 6, train loss 0.2787, train acc 0.899, test loss 0.3268, test acc 0.879, time 14.1 sec
epoch 7, train loss 0.2633, train acc 0.903, test loss 0.3374, test acc 0.875, time 14.1 sec
epoch 8, train loss 0.2460, train acc 0.911, test loss 0.3295, test acc 0.878, time 13.8 sec
epoch 9, train loss 0.2320, train acc 0.915, test loss 0.3201, test acc 0.883, time 14.1 sec
epoch 10, train loss 0.2170, train acc 0.921, test loss 0.3207, test acc 0.881, time 14.1 sec
